# 1. Import Libraries 

In [2]:
import os
import glob
import sys
import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import seaborn as sns
import altair as alt

from skimage import img_as_ubyte, io, color
import cv2

from sklearn.model_selection import train_test_split

#Data balancing SMOTE - imblearn
from imblearn.over_sampling import SMOTE

#PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F

#Skorch
from skorch import NeuralNetClassifier, NeuralNet, NeuralNetRegressor
from skorch.callbacks import EarlyStopping, LRScheduler, EpochScoring,ProgressBar, Checkpoint
from skorch.dataset import CVSplit, ValidSplit
from skorch.callbacks import TrainEndCheckpoint

#model performance
from sklearn.metrics import classification_report,confusion_matrix,ConfusionMatrixDisplay,top_k_accuracy_score, balanced_accuracy_score, f1_score, recall_score

#Scaling
from sklearn.preprocessing import StandardScaler, MinMaxScaler

#Distnce Matrix
from scipy.spatial import distance_matrix, distance

#Supress warnings
import warnings
from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings(action='ignore')

from torch.utils.data import Dataset, DataLoader
#from sklearnex import patch_sklearn
#patch_sklearn()


# 2. Import Data

In [4]:
continous_vector_KG04_df = pd.read_csv("C:/Dissertation/Data/Processed Data/Custom_Animal_Attribute_KG_continous_vectors.csv")
continous_vector_KG04_df.drop(['Unnamed: 0'],axis=1, inplace = True)
continous_vector_KG04_df.rename(columns={"index": "class"}, inplace = True)
continous_vector_KG04_df.head()

,class,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610,611,612,613,614,615,616,617,618,619,620,621,622,623,624,625,626,627,628,629,630,631,632,633,634,635,636,637,638,639,640,641,642,643,644,645,646,647,648,649,650,651,652,653,654,655,656,657,658,659,660,661,662,663,664,665,666,667,668,669,670,671,672,673,674,675,676,677,678,679
0,antelope,0.460994,-0.063934,0.420526,-0.279000,-0.065227,0.277337,-0.068888,-0.032580,-0.094497,0.038179,0.236784,-0.322386,-0.103949,0.107141,-0.151514,0.035952,-0.275462,0.037563,0.103750,0.008782,0.046787,-0.021176,0.378095,0.093829,-0.062982,-0.133867,-0.055049,0.029432,0.017505,-0.046846,0.304332,-0.165837,-0.034567,0.316529,-0.004667,0.271108,-0.098861,0.108340,0.143083,-0.232294,0.139438,0.330613,0.225732,0.058820,0.060985,0.257397,0.055370,0.104763,-0.364362,0.118215,0.121255,-0.209652,0.078562,-0.282409,-0.183032,-0.019264,0.029163,0.048381,-0.103341,0.039271,0.364192,0.089079,0.041127,0.139229,0.193123,0.067591,0.059983,-0.083476,-0.061693,-0.501603,0.179091,-0.109476,-0.115007,-0.003436,-0.083498,0.174076,0.009389,0.215875,0.008448,-0.171699,0.124560,0.333008,-0.353874,-0.100023,-0.082624,0.230158,0.069488,0.189537,-0.040320,0.055358,-0.216844,-0.050265,-0.065202,0.143897,-0.014838,-0.368200,0.109446,-0.168831,-0.374319,-0.212464,-0.015157,-0.022182,0.201586,-0.117472,-0.146512,-0.346831,-0.035460,0.021918,0.068003,0.178094,-0.125352,-0.216272,0.454746,0.175763,-0.174145,-0.074190,0.048322,-0.141452,-0.153340,0.009596,0.108646,-0.046925,0.004769,0.056757,-0.135249,-0.025580,-0.061640,-0.082165,-0.057556,0.057967,-0.120897,-0.125779,0.058547,-0.171336,0.148606,-0.162540,-0.049027,0.275602,0.102090,-0.174689,0.166637,-0.046710,-0.099178,0.243635,-0.1

In [5]:
vector = continous_vector_KG03_df.iloc[:,1:]
classes = continous_vector_KG03_df['class']

In [6]:
from scipy.spatial import distance_matrix
original_dist_df = pd.DataFrame(distance_matrix(vector, vector), index=classes, columns=classes)

In [7]:
original_dist_df.head()

class,antelope,grizzly+bear,killer+whale,beaver,dalmatian,persian+cat,horse,german+shepherd,blue+whale,siamese+cat,skunk,mole,tiger,hippopotamus,leopard,moose,spider+monkey,humpback+whale,elephant,gorilla,ox,fox,sheep,seal,chimpanzee,hamster,squirrel,rhinoceros,rabbit,bat,giraffe,wolf,chihuahua,rat,weasel,otter,buffalo,zebra,giant+panda,deer,bobcat,pig,lion,mouse,polar+bear,collie,walrus,raccoon,cow,dolphin
class,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
antelope,0.000000,4.475147,4.785332,4.466352,3.706341,4.855303,3.022426,3.881333,4.503949,4.278894,4.323319,4.014368,4.102195,4.555057,3.780823,2.720055,3.553057,4.632040,3.942029,3.792683,3.653497,3.705903,3.882038,4.671608,3.502585,4.351878,4.064013,4.077577,3.671594,4.520874,3.142946,3.945274,4.958944,4.625452,4.317141,4.767907,3.188224,2.402857,4.502051,2.424909,3.898377,3.949071,4.081341,4.023954,4.548110,4.037129,5.181055,4.314612,3.212749,4.209672
grizzly+bear,4.475147,0.000000,4.277975,3.355941,4.130883,4.019241,4.447348,3.674225,4.063071,3.291986,3.694394,3.718093,3.355016,4.071074,2.794732,3.702606,4.059043,4.095827,4.281923,3.521873,4.275166,2.499726,4.427313,4.401283,3.844673,4.258632,3.932101,4.104347,4.399241,3.346857,4.456888,2.835355,4.121516,3.595551,3.543559,3.905208,3.443333,4.166017,3.585686,4.197005,2.499697,3.803216,3.315773,4.484325,3.282614,3.907288,4.347752,3.229963,4.276749,4.801185
killer+whale,4.785332,4.277975,0.000000,4.157940,3.796187,4.556841,4.281696,4.152476,3.284639,4.314590,5.052980,4.934432,3.834035,4.806337,4.019190,4.854537,4.915397,3.218567,4.858674,4.121428,4.820058,4.297992,5.301838,3.059612,4.385665,4.948060,5.216421,4.882133,4.745923,4.456595,4.759373,4.065134,4.905254,4.347921,4.584285,3.906879,4.592673,4.044921,4.666327,4.710442,4.365487,4.267631,4.185004,5.188602,3.995941,4.099273,3.783726,4.328366,4.529588,3.120091
beaver,4.466352,3.355941,4.157940,0.000000,3.949857,3.840292,4.373712,4.416741,3.886229,3.915067,3.926013,3.622331,3.681482,3.904660,3.660779,4.256903,4.060329,4.072510,4.636026,3.972390,4.693330,3.272307,4.977572,4.124334,4.192647,3.898365,3.651700,4.798162,4.017279,3.920119,4.857598,3.837480,4.460308,3.937390,4.217331,3.316509,4.448924,4.427031,4.076878,4.189355,3.751670,4.527332,4.123857,4.019940,4.047201,4.239587,3.647936,3.815997,4.637570,3.900028
dalmatian,3.706341,4.130883,3.796187,3.949857,0.000000,3.023724,2.860076,2.645770,3.904207,2.764750,4.407000,4.423380,3.450030,4.190258,3.388633,4.179542,3.647000,4.158674,4.056381,3.545272,3.618314,3.799438,4.265558,3.823958,3.378370,4.273767,4.245620,4.294558,3.955084,4.342340,3.906465,3.602110,3.242510,4.248364,4.000571,4.225907,3.817950,2.940352,3.789933,3.591052,3.699830,3.373193,4.116394,4.385122,4.348960,2.187442,4.546041,3.759594,3.399199,3.080631


In [8]:
class VectorDataset(Dataset):

    def __init__(self):
        # Initialize data, download, etc.
        # read with numpy or pandas
        data = continous_vector_KG03_df.iloc[:,1:681].astype(np.float32).values
        self.n_samples = data.shape[0]
       
        # here the first column is the class label, the rest are the features
        self.x_data = torch.from_numpy(data) # size [n_samples, n_features]
        self.y_data = torch.from_numpy(data) # size [n_samples, 1]

    # support indexing such that dataset[i] can be used to get i-th sample
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    # we can call len(dataset) to return the size
    def __len__(self):
        return self.n_samples

dataset = VectorDataset()

# 3. Set up Autoencoder Architecture

### 3.1 Straight-Through Estimator Architecture

In [9]:
#https://discuss.pytorch.org/t/binary-activation-function-with-pytorch/56674/4
#https://hassanaskary.com/python/pytorch/deep%20learning/2020/09/19/intuitive-explanation-of-straight-through-estimators.html#:~:text=A%20straight-through%20estimator%20is,function%20was%20an%20identity%20function.
class STEFunction(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input):
        return (input > 0).float()

    @staticmethod
    def backward(ctx, grad_output):
        return F.hardtanh(grad_output)

class StraightThroughEstimator(nn.Module):
    def __init__(self):
        super(StraightThroughEstimator, self).__init__()

    def forward(self, x):
        x = STEFunction.apply(x)
        return x

### 3.1 Define DNN for projection network

In [28]:
class Encoder(nn.Module):
    def __init__(self, num_units=300):
        super().__init__()
        self.num_units = num_units
        
        self.encode = nn.Sequential(
            nn.Linear(680, 680),
            nn.ReLU(),
            nn.Linear(680, self.num_units),
            StraightThroughEstimator(), 
        )
        
    def forward(self, X):
        encoded = self.encode(X)
        return encoded

In [29]:
class Decoder(nn.Module):
    def __init__(self, num_units):
        super().__init__()
        self.num_units = num_units
        
        self.decode = nn.Sequential(
            nn.Linear(self.num_units, 680),
            nn.ReLU(),
            nn.Linear(680, 680),
            nn.Tanh(),
        )
        
    def forward(self, X):
        decoded = self.decode(X)
        return decoded

In [30]:
class AutoEncoder(nn.Module):
    def __init__(self, num_units):
        super().__init__()
        self.num_units = num_units

        self.encoder = Encoder(num_units=self.num_units)
        self.decoder = Decoder(num_units=self.num_units)
        
    def forward(self, X):
        encoded = self.encoder(X)
        decoded = self.decoder(encoded)
        return decoded, encoded  # <- return a tuple of two values

In [31]:
class ContinoustoBinaryAutoencoder(nn.Module):
    def __init__(self):
        super(ContinoustoBinaryAutoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(680, 340),
            nn.ReLU(),
            nn.Linear(340, 170),
            nn.ReLU(),
            nn.Linear(170, 85), 
            nn.ReLU(),
            StraightThroughEstimator(),            
            )       
        
        
        self.decoder = nn.Sequential(
            nn.Linear(85, 170),
            nn.ReLU(),
            nn.Linear(170, 340),
            nn.ReLU(),
            nn.Linear(340, 680),
            nn.Tanh()
        )

    def forward(self, x):
        encoding = self.encoder(x)
        decoding  = self.decoder(encoding)
        return decoding

In [32]:
model = AutoEncoder(num_units=680)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(),
                             lr=0.0001)                           


In [33]:
from tqdm import tqdm
dataloader = DataLoader(dataset=dataset,
                          batch_size=3,
                          shuffle=True)
epoch = 10
for e in range(epoch):
    print(f'Starting epoch {e} of {epoch}')
    for X, y in tqdm(dataloader):
        optimizer.zero_grad()
        reconstruction = model(X)
        loss = criterion(reconstruction, X)
        loss.backward()
        optimizer.step()        
    print(f'Loss: {loss.item()}')
    

Starting epoch 0 of 10


  0%|          | 0/17 [00:00<?, ?it/s]


AttributeError: 'tuple' object has no attribute 'size'

In [21]:
with torch.no_grad():
    recon_vector = model(torch.from_numpy(continous_vector_KG03_df.iloc[:,1:681].astype(np.float32).values)).detach().numpy()
        
recon_df = pd.DataFrame(recon_vector, index=classes)
recon_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610,611,612,613,614,615,616,617,618,619,620,621,622,623,624,625,626,627,628,629,630,631,632,633,634,635,636,637,638,639,640,641,642,643,644,645,646,647,648,649,650,651,652,653,654,655,656,657,658,659,660,661,662,663,664,665,666,667,668,669,670,671,672,673,674,675,676,677,678,679
class,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
antelope,-0.025803,0.034234,-0.000305,0.002679,-0.002155,-0.030659,-0.002250,-0.020038,-0.003146,-0.002506,-0.001494,0.003207,0.007664,0.015842,-0.024266,0.006500,-0.004178,-0.002354,-0.028058,0.015604,0.004455,0.002747,-0.032462,0.035120,0.021163,0.001470,0.015645,0.022717,-0.012205,-0.002291,0.015962,-0.000078,-0.012834,0.000986,-0.036367,0.008301,0.006414,0.000278,0.009226,-0.008858,-0.006917,-0.010729,-0.023454,0.003255,-0.009179,0.003215,-0.019731,0.053090,-0.010843,0.005057,-0.016681,0.027689,0.003775,0.011431,0.004716,-0.020386,0.009840,0.019156,0.022456,0.012486,0.011557,0.023065,-0.005204,-0.006613,0.006713,-0.012098,0.014430,0.029464,-0.012635,0.009197,0.016937,-0.008459,0.029193,-0.0

In [22]:
activation = {}
def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook

In [25]:
with torch.no_grad():
    model.encoder.register_forward_hook(get_activation('encoder'))
    output = model(torch.from_numpy(continous_vector_KG03_df.iloc[:,1:681].astype(np.float32).values)).detach().numpy()

    binary_vector_df = pd.DataFrame(activation['encoder'].detach().numpy(), index=classes)
    binary_vector_df = binary_vector_df.astype(int)
    binary_vector_df
binary_vector_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84
class,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
antelope,1,1,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,1,1,0,0,0,0,1,0,0,0,0,1,1,0,1,1,1,1,0,0,0,1,1,1,0,1,0,1,0,1,1,1,1,1,0,1,0,1,1,1,0,0,1,0,0,0,0,0,1,1,1,0,0,0,0,1,0,1,0,1,0,1
grizzly+bear,1,1,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,1,1,0,0,0,0,1,0,0,0,0,1,1,0,1,1,1,1,0,0,0,1,1,1,0,1,0,1,0,1,1,1,1,1,0,1,0,1,1,1,0,0,1,0,0,0,0,0,1,1,1,0,0,0,0,1,0,1,0,1,0,1
killer+whale,1,1,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,1,1,0,0,0,0,1,0,0,0,0,1,1,0,1,1,1,1,0,0,0,1,1,1,0,1,0,1,0,1,1,1,1,1,0,1,0,1,1,1,0,0,1,0,0,0,0,0,1,1,1,0,0,0,0,1,0,1,0,1,0,1
beaver,1,1,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,1,1,0,0,0,0,1,0,0,0,0,1,1,0,1,1,1,1,0,0,0,1,1,1,0,1,0,1,0,1,1,1,1,1,0,1,0,1,1,1,0,0,1,0,0,0,0,0,1,1,1,0,0,0,0,1,0,1,0,1,0,1
dalmatian,1,1,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,1,1,0,0,0,0,1,0,0,0,0,1,1,0,1,1,1,1,0,0,0,1,1,1,0,1,0,1,0,1,1,1,1,1,0,1,0,1,1,1,0,0,1,0,0,0,0,0,1,1,1,0,0,0,0,1,0,1,0,1,0,1
persian+cat,1,1,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,1,1,0,0,0,0,1,0,0,0,0,1,1,0,1,1,1,1,0,0,0,1,1,1,0,1,0,1,0,1,1,1,1,1,0,1,0,1,1,1,0,0,1,0,0,0,0,0,1,1,1,0,0,0,0,1,0,1,0,1,0,1
horse,1,1,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,1,1,0,0,0,0,1,0,0,0,0,1,1,0,1,1,1,1,0,0,0,1,1,1,0,1,0,1,0,1,1,1,1,1,0,1,0,1,1,1,0,0,1,0,0,0,0,0,1,1,1,0,0,0,0,1,0,1,0,1,0,1
german+shepherd,1,1,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,1,1,0,0,0,0,1,0,0,0,0,1,1,0,1,1,1,1,0,0,0,1,1,1,0,1,0,1,0,1,1,1,1,1,0,1,0,1,1,1,0,0,1,0,0,0,0,0,1,1,1,0,0,0,0,1,0,1,0,1,0,1
blue+whale,1,1,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,1,1,0,0,0,0,1,0,0,0,0,1,1,0,1,1,1,1,0,0,0,1,1,1,0,1,0,0,0,1,1,1,1,1,0,1,0,1,1,1,0,0,1,0,0,0,0,0,1,1,1,0,0,0,0,1,0,1,0,1,0,1


In [ ]:
import argparse
import optuna
from optuna.integration import SkorchPruningCallback

class ContinoustoBinaryAutoencoder(nn.Module):
    def __init__(self):
        super(ContinoustoBinaryAutoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(85, 48),
            nn.ReLU(),
            nn.Linear(48, 24),
            nn.ReLU(),
            nn.Linear(24, 12), 
            nn.ReLU(),
            StraightThroughEstimator(),            
            )       
        
        
        self.decoder = nn.Sequential(
            nn.Linear(12, 24),
            nn.ReLU(),
            nn.Linear(24, 48),
            nn.ReLU(),
            nn.Linear(48, 85),
            nn.Tanh()
        )

    def forward(self, x, encode=False, decode=False):
        if encode:
            x = self.encoder(x)
        elif decode:
            x = self.decoder(x)
        else:
            encoding = self.encoder(x)
            x = self.decoder(encoding)
        return x
    

def objective(trial: optuna.Trial):
    lr = trial.suggest_loguniform('lr', .00001, 1)
    bs = trial.suggest_int("batch_size", 2, 8)
    ep = trial.suggest_int("max_epoch", 5,20)
    wd = trial.suggest_loguniform('optimizer__weight_decay', .00001, .1)
    
    dataloader = DataLoader(dataset=dataset,
                          batch_size=bs,
                          shuffle=True)
    
    model = ContinoustoBinaryAutoencoder()
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(),
                             lr=lr, 
                             weight_decay=wd)      
    
    epoch = ep
    for e in range(epoch):
        #print(f'Starting epoch {e} of {epoch}')
        for X, y in dataloader:
            optimizer.zero_grad()
            reconstruction = model(X)
            loss = criterion(reconstruction, X)
            loss.backward()
            optimizer.step()      
    
    with torch.no_grad():
        recon_vector = model(torch.from_numpy(continous_vector_KG03_df.iloc[:,1:86].astype(np.float32).values))

    recon_dist_df = pd.DataFrame(distance_matrix(recon_vector, recon_vector), index=classes, columns=classes)    
   
    diff_df = original_dist_df - recon_dist_df
    mean_diff = np.median(np.mean(diff_df).values)
    
    return mean_diff

In [ ]:
if __name__ == "__main__":
    parser = argparse.ArgumentParser(description="skorch example.")
    parser.add_argument(
        "--pruning",
        "-p",
        action="store_true",
        help="Activate the pruning feature. `MedianPruner` stops unpromising "
        "trials at the early stages of training.",
    )
    args, unknown = parser.parse_known_args()

    pruner = optuna.pruners.MedianPruner() if args.pruning else optuna.pruners.NopPruner()

    study = optuna.create_study(direction="minimize", pruner=pruner)
    study.optimize(objective, n_trials=200, timeout=1000)

    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))